# 2_ML_algorithms_for_supervised_learning

In this notebook I'm going to implement the diferent algorithms to supervised learning 

# Imports  

In [34]:
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
from IPython.display import display

import seaborn as sns
import h2o
from h2o.estimators.xgboost import H2OXGBoostEstimator
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.impute import SimpleImputer
from joblib import dump, load
from sklearn.model_selection import train_test_split
from imblearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders.woe import WOEEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

sys.path.append('..')
from utilities.helpers_data_cleaning import (distincts_in_col,
                                             convert_categorical,
                                             nan_in_col,
                                             read_feats,
                                             generate_synthetic_train_data,
                                             obtain_features_label,
                                             fill_categorical,
                                             split_data,
                                             plot_pca,
                                             pca_features,
                                             elbow_Kmeans,
                                             cast_categorical_h2o)

# Constants

In [2]:
PATH_DATA_GERMANY = Path('../data/german_poblation/')
PATH_DATA_CUSTOMER = Path('../data/customers/')
PATH_DATA_MAILOUT = Path('../data/mailout/')
PATH_FEATS = Path("../data/feats/DIAS Attributes - Values 2017.xlsx")
PATH_SAVE = Path("../data/data_processed")
PAT_SAVE_PIPELINES = Path("../data/pipelines")
pipeline_pca = load(PAT_SAVE_PIPELINES / 'pipeline_pca_full_variables.pkl')
pipeline_kmeans = load(PAT_SAVE_PIPELINES / 'pipeline_kmeans_full_variables.pkl')

## Obtain features from models previously trained

In [3]:
h2o.init()

Proxy is defined in the environment: HTTP_PROXY. This may interfere with your H2O Connection.
Proxy is defined in the environment: http_proxy. This may interfere with your H2O Connection.


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2019-01-15; OpenJDK Runtime Environment 18.9 (build 11.0.2+9); OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from /home/jguijarh/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpww2ijttx
  JVM stdout: /tmp/tmpww2ijttx/h2o_jguijarh_started_from_python.out
  JVM stderr: /tmp/tmpww2ijttx/h2o_jguijarh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,1 month and 3 days
H2O_cluster_name:,H2O_from_python_jguijarh_u2nrif
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.746 Gb
H2O_cluster_total_cores:,5
H2O_cluster_allowed_cores:,5
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
xgb_model = h2o.load_model('../data/models/model_less_features_with_cluster/model_with_cluster')
popu_data = h2o.import_file(path='../data/data_processed/synthetic_df_with_clusters_ver_2.csv')
nullity_vars = list(xgb_model.actual_params['ignored_columns'])
exclude_feats = nullity_vars + ['LNR', 'target_customer']

features_with_cluster = [feat for feat in popu_data.columns if feat not in exclude_feats]
label_with_cluster = ['target_customer']

label_train = label_with_cluster
features_train = features_with_cluster

h2o.remove_all()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
h2o.cluster().shutdown()

H2O session _sid_b645 closed.


In [6]:
features_dict = read_feats(PATH_FEATS)

In [7]:
df_mailout_train = pd.read_csv(PATH_DATA_MAILOUT /
                               'Udacity_MAILOUT_052018_TRAIN.csv',
                               dtype= features_dict,
                               na_values=[-1,'X','XX'],
                               sep=';').drop(columns = ['Unnamed: 0'])
df_mailout_train = convert_categorical(df_mailout_train)
df_mailout_train = fill_categorical(df_mailout_train)

df_mailout_test = pd.read_csv(PATH_DATA_MAILOUT /
                              'Udacity_MAILOUT_052018_TEST.csv',
                              dtype= features_dict,
                              na_values=[-1,'X','XX'],
                              sep=';').drop(columns = ['Unnamed: 0'])
df_mailout_test = convert_categorical(df_mailout_test)
df_mailout_test = fill_categorical(df_mailout_test)
df_mailout_train = df_mailout_train.rename(columns={'RESPONSE': label_train[0]})

features, label = obtain_features_label(df_mailout_train)

# Obtain the clusters
X_mailout_train = (df_mailout_train[features]
                   .select_dtypes(include='number')
                   .reset_index(drop=True).copy())
df_mailout_train['CLUSTER'] = pipeline_kmeans.predict(X_mailout_train)

X_mailout_test = (df_mailout_test[features]
                     .select_dtypes(include='number')
                     .reset_index(drop=True).copy())
df_mailout_test['CLUSTER'] = pipeline_kmeans.predict(X_mailout_test)

# Predict the cluster and insert the new feat in the dictionary
df_mailout_train['CLUSTER'] = df_mailout_train['CLUSTER'].astype('category')
df_mailout_test['CLUSTER'] = df_mailout_test['CLUSTER'].astype('category')
features_dict['CLUSTER'] = 'category'
cat_feats = df_mailout_train.select_dtypes(include='category').columns

#save the processed data
df_mailout_train.to_csv(PATH_SAVE / 'df_mailout_train_processed.csv', index=False)
df_mailout_test.to_csv(PATH_SAVE / 'df_mailout_test_processed.csv', index=False)

## Split the train data

First, I just going to train the model with mail train data

In [336]:
X = df_mailout_train[features_train]
y = df_mailout_train[label_train[0]]

cat_feats = X.select_dtypes(include=['category','object']).columns

X_train, X_test, y_train, y_test = (
    train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=1993,
        stratify=y))

-----------------------------
# XGBoost

## Define the pipeline to train the model
In this pipeline I do three main steps:
    - An oversample of the data, to increase the ratio that positive cases have in the data
    - A weights of evidence to encode the categorical data.

In [340]:
param_xgb = {
    "ntrees" : 40,
    "max_depth" : 8,
    "learn_rate" : 0.01,
    "sample_rate" : 0.8,
    "col_sample_rate_per_tree" : 0.8,
    "min_rows" : 3,
    "random_state": 1993,
    "stopping_metric": 'AUC',
    "n_jobs":-1,
#     "reg_alpha":0.1
    }
    
pipeline_encoder_and_xgb = make_pipeline(
    RandomOverSampler(sampling_strategy=.02, random_state=1993),
    WOEEncoder(cols=cat_feats),
    XGBClassifier(**param_xgb)
)

In [341]:
pipeline_encoder_and_xgb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('randomoversampler',
                 RandomOverSampler(random_state=1993, sampling_strategy=0.02)),
                ('woeencoder',
                 WOEEncoder(cols=Index(['ALTER_HH', 'BALLRAUM', 'CAMEO_DEU_2015', 'CAMEO_DEUG_2015',
       'CJT_GESAMTTYP', 'D19_BANKEN_DATUM', 'D19_BANKEN_OFFLINE_DATUM',
       'D19_BANKEN_ONLINE_DATUM', 'D19_GESAMT_DATUM',
       'D19_GESAMT_OFFLINE_DATUM',
       ...
       'SEMIO_TR...
                               interaction_constraints=None, learn_rate=0.01,
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=8, min_child_weight=1, min_rows=3,
                               missing=nan, monotone_constraints=None,
                               n_estimators=100, n_jobs=-1, ntrees=40,
                               num_parallel_tree=1, objective='binary:logistic',
                               random_state=1993, reg_alpha=0, reg_lambda=1,
      

In [343]:
auc_xgb = roc_auc_score(
    y_test, pipeline_encoder_and_xgb.predict_proba(X_test)[:, 1]
)
auc_xgb

0.6812750243991366

In [344]:
df_kaggle_submission_xgb = pd.DataFrame(dict(LNR=df_mailout_test['LNR'],
                                                  RESPONSE=pipeline_encoder_and_xgb.predict_proba(df_mailout_test[features_train])[:,1]))

df_kaggle_submission_xgb.to_csv('../data/kaggle_competition/prediction_xgboost_1.csv',index=False)

--------------
# CatBoost

## Define the pipeline
In this pipeline, I define two main steps:
    - 

In [345]:
params_catboost = dict(num_trees=15,
                       max_depth=3,
                       min_child_samples=25,
                       one_hot_max_size=4,
                       cat_features=cat_feats,
                       random_state=1993,
                       eval_metric='AUC',
                       verbose=0)

pipeline_encoder_catboost = make_pipeline(
    RandomOverSampler(sampling_strategy=.02, random_state=1993),
    CatBoostClassifier(**params_catboost)
)

pipeline_encoder_catboost.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('randomoversampler',
                 RandomOverSampler(random_state=1993, sampling_strategy=0.02)),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7fa10f2e95d0>)],
         verbose=False)

In [346]:
auc_catboost_oversampling = roc_auc_score(
    y_test, 
    pipeline_encoder_catboost.predict_proba(X_test)[:, 1])
auc_catboost_oversampling

0.7812842504963196

In [348]:
df_kaggle_submission_catboost = pd.DataFrame(dict(LNR=df_mailout_test['LNR'],
                                                  RESPONSE=pipeline_encoder_catboost.predict_proba(df_mailout_test[features_train])[:,1]))

df_kaggle_submission_catboost.to_csv('../data/kaggle_competition/prediction_catboost_1.csv',index=False)

In [202]:
varimp_cat = dict(zip(pipeline_encoder_catboost.steps[-1][1].feature_names_,
    pipeline_encoder_catboost.steps[-1][1].feature_importances_))

# LightGBM

Define the lightgbm pipeline

In [349]:
param_lgb = {
    "ntrees" : 40,
    "max_depth" : 8,
    "learn_rate" : 0.01,
    "sample_rate" : 0.8,
    "col_sample_rate_per_tree" : 0.8,
    "min_rows" : 3,
    "random_state": 1993,
    "stopping_metric": 'AUC',
    "n_jobs":-1
#     "reg_alpha":0.1
    }
    
pipeline_encoder_and_lgb = make_pipeline(
    WOEEncoder(cols=cat_feats),
    lgb.LGBMClassifier(**param_lgb)
)

In [ ]:
pipeline_encoder_and_lgb.fit(X_train, y_train)

In [271]:
auc_lgb = roc_auc_score(
    y_test, pipeline_encoder_and_lgb.predict_proba(X_test)[:, 1]
)
auc_lgb

0.6969944043164795

In [ ]:
df_kaggle_submission_lgboost = pd.DataFrame(dict(LNR=df_mailout_test['LNR'],
                                                  RESPONSE=pipeline_encoder_and_lgb.predict_proba(df_mailout_test[features_train])[:,1]))

df_kaggle_submission_lgboost.to_csv('../data/kaggle_competition/prediction_lgboost_1.csv',index=False)

# Train with Population data mixed with mailout campaign train

Define the data that I'm going to use in this section. Te data will be:
- 15% of population data that i have used in the first section of this project.
- MAILOUT train data.

I read the data that contein population data and get a sample of 15%

In [24]:
df_population = (pd.read_csv('../data/data_processed/synthetic_df_with_clusters_ver_2.csv',
                             dtype= features_dict)
                 .sample(frac=0.15)
                 .reset_index(drop=True))

# df_population = (pd.read_csv('../data/data_processed/synthetic_df_with_clusters_ver_2.csv',
#                              dtype= features_dict)
#                  .sample(frac=0.20)
#                  .reset_index(drop=True))

Create the data set to future data treatment

In [25]:
df_train_composed = pd.concat([df_population, df_mailout_train]).reset_index(drop=True)

In [26]:
df_train_composed = df_train_composed.reset_index(drop=True)

Split the data

In [28]:
X = df_train_composed[features_train]
y = df_train_composed[label_train[0]]

cat_feats = X.select_dtypes(include=['category','object']).columns

X_train, X_test, y_train, y_test = (
    train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=1993,
        stratify=y))

## XGBoost mixed data

Define the XGBoost and the pipeline to the data preprocessing

In [29]:
param_xgb = {
    "ntrees" : 40,
    "max_depth" : 8,
    "learn_rate" : 0.01,
    "sample_rate" : 0.8,
    "col_sample_rate_per_tree" : 0.8,
    "min_rows" : 3,
    "random_state": 1993,
    "stopping_metric": 'AUC',
    "n_jobs":-1,
#     "reg_alpha":0.1
    }
    
pipeline_encoder_and_xgb_mixed = make_pipeline(
    WOEEncoder(cols=cat_feats),
    XGBClassifier(**param_xgb)
)

NameError: name 'XGBClassifier' is not defined

In [ ]:
pipeline_encoder_and_xgb_mixed.fit(X_train, y_train)

In [308]:
auc_xgb = roc_auc_score(
    y_test, pipeline_encoder_and_xgb_mixed.predict_proba(X_test)[:, 1]
)
auc_xgb

0.9537283622549791

In [313]:
df_kaggle_submission_xgb = pd.DataFrame(dict(LNR=df_mailout_test['LNR'],
                                                  RESPONSE=pipeline_encoder_and_xgb_mixed.predict_proba(df_mailout_test[features_train])[:,1]))

df_kaggle_submission_xgb.to_csv('../data/kaggle_competition/prediction_xgboost_mixed_data_1.csv',index=False)

In [314]:
cv_xgb=None

In [310]:
cv_lgb = cross_val_score_catboost_oversampling = None

--------------
# CatBoost with mixed data

## Define the pipeline
In this pipeline, I define two main steps:
    - 

In [14]:
params_catboost = dict(num_trees=40,
                       max_depth=4,
                       min_child_samples=25,
                       one_hot_max_size=4,
                       cat_features=cat_feats,
                       random_state=1993,
                       eval_metric='AUC',
                       verbose=0)


pipeline_encoder_catboost_mixed = make_pipeline(
    CatBoostClassifier(**params_catboost)
)

pipeline_encoder_catboost_mixed.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7f27636ddcd0>)],
         verbose=False)

In [15]:
auc_catboost_oversampling_mixed = roc_auc_score(
    y_test, 
    pipeline_encoder_catboost_mixed.predict_proba(X_test)[:, 1])
auc_catboost_oversampling_mixed

0.92073899054021

In [16]:
df_kaggle_submission_catboost_mixed = pd.DataFrame(dict(LNR=df_mailout_test['LNR'],
                                                  RESPONSE=pipeline_encoder_catboost_mixed.predict_proba(df_mailout_test[features_train])[:,1]))

df_kaggle_submission_catboost_mixed.to_csv('../data/kaggle_competition/prediction_catboost_mixed_data_2.csv',index=False)

Save the importance of variables inside the model

In [202]:
varimp_cat = dict(zip(pipeline_encoder_catboost.steps[-1][1].feature_names_,
                      pipeline_encoder_catboost.steps[-1][1].feature_importances_))

# LightGBM with mixed data

Define the lightgbm pipeline

In [35]:
param_lgb = { 
    "ntrees" : 50,
    "max_depth" : 8,
    "learn_rate" : 0.01,
    "sample_rate" : 0.8,
    "col_sample_rate_per_tree" : 0.8,
    "min_rows" : 3,
    "random_state": 1993,
    "stopping_metric": 'AUC',
    "n_jobs":-1
    }

pipeline_encoder_and_lgb_mixed = make_pipeline(
    WOEEncoder(cols=cat_feats),
    lgb.LGBMClassifier(**param_lgb)
)

In [36]:
pipeline_encoder_and_lgb_mixed.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('woeencoder',
                 WOEEncoder(cols=Index(['ALTER_HH', 'BALLRAUM', 'CAMEO_DEU_2015', 'CAMEO_DEUG_2015',
       'CJT_GESAMTTYP', 'D19_BANKEN_DATUM', 'D19_BANKEN_OFFLINE_DATUM',
       'D19_BANKEN_ONLINE_DATUM', 'D19_GESAMT_DATUM',
       'D19_GESAMT_OFFLINE_DATUM',
       ...
       'SEMIO_TRADV', 'SEMIO_VERT', 'VERS_TYP', 'W_KEIT_KIND_HH',
       'WOHNDAUER_2008', 'WOHNLAGE', 'ZABEOTY...
                                learn_rate=0.01, learning_rate=0.1,
                                max_depth=15, min_child_samples=20,
                                min_child_weight=0.001, min_rows=3,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                ntrees=80, num_leaves=31, objective=None,
                                random_state=1993, reg_alpha=0.0,
                                reg_lambda=0.0, sample_rate=0.8, silent=True,
                                stopping_metric='AUC', s

In [32]:
auc_lgb = roc_auc_score(
    y_test, pipeline_encoder_and_lgb_mixed.predict_proba(X_test)[:, 1]
)
auc_lgb

0.9609141525552982

Save teh data to submit on kaggle competition

In [37]:
df_kaggle_submission_lgboost = pd.DataFrame(dict(LNR=df_mailout_test['LNR'],
                                                  RESPONSE=pipeline_encoder_and_lgb_mixed.predict_proba(df_mailout_test[features_train])[:,1]))

df_kaggle_submission_lgboost.to_csv('../data/kaggle_competition/prediction_lgboost_mixed_8.csv',index=False)